<div style='background-image: url("../share/images/header.svg") ; padding: 0px ; background-size: cover ; border-radius: 5px ; height: 250px'>
    <div style="float: right ; margin: 50px ; padding: 20px ; background: rgba(255 , 255 , 255 , 0.7) ; width: 50% ; height: 150px">
        <div style="position: relative ; top: 50% ; transform: translatey(-50%)">
            <div style="font-size: xx-large ; font-weight: 900 ; color: rgba(0 , 0 , 0 , 0.8) ; line-height: 100%">ObsPy Tutorial</div>
            <div style="font-size: large ; padding-top: 20px ; color: rgba(0 , 0 , 0 , 0.5)">Handling Event Metadata</div>
        </div>
    </div>
</div>

Seismo-Live: http://seismo-live.org

##### Authors:
* Lion Krischer ([@krischer](https://github.com/krischer))
* Tobias Megies ([@megies](https://github.com/megies))
---

![](images/obspy_logo_full_524x179px.png)

In [1]:
%matplotlib inline
#from __future__ import print_function
import matplotlib.pyplot as plt
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = 12, 8

- for event metadata, the de-facto standard is [QuakeML (an xml document structure)](https://quake.ethz.ch/quakeml/)
- QuakeML files can be read using **`read_events()`**

In [2]:
from obspy import read_events

catalog = read_events("./data/event_tohoku_with_big_aftershocks.xml")
print(catalog)

5 Event(s) in Catalog:
2011-03-11T05:46:23.200000Z | +38.296, +142.498 | 9.1  MW
2011-03-11T06:15:37.570000Z | +36.227, +141.088 | 7.9  MW
2011-03-11T06:25:50.740000Z | +38.051, +144.630 | 7.6  MW
2011-04-07T14:32:43.290000Z | +38.276, +141.588 | 7.1  MW
2011-07-10T00:57:10.800000Z | +38.034, +143.264 | 7.0  MW


- **`read_events()`** function returns a **`Catalog`** object, which is
a collection of **`Event`** objects.

In [3]:
print(type(catalog))
print(type(catalog[0]))

<class 'obspy.core.event.catalog.Catalog'>
<class 'obspy.core.event.event.Event'>


In [3]:
event = catalog[2]
print(event)

Event:	2011-03-11T06:25:50.740000Z | +38.051, +144.630 | 7.6  MW

	            resource_id: ResourceIdentifier(id="smi:service.iris.edu/fdsnws/event/1/query?eventid=3279409")
	             event_type: 'earthquake'
	    preferred_origin_id: ResourceIdentifier(id="smi:service.iris.edu/fdsnws/event/1/query?originid=9933564")
	 preferred_magnitude_id: ResourceIdentifier(id="smi:service.iris.edu/fdsnws/event/1/query?magnitudeid=16642722")
	                   ---------
	     event_descriptions: 1 Elements
	                origins: 1 Elements
	             magnitudes: 1 Elements


- Event objects are again collections of other resources.
- the nested ObsPy Event class structure (Catalog/Event/Origin/Magnitude/FocalMechanism/...) is closely modelled after QuakeML
<img src="images/Event.svg" width=90%>

In [4]:
from obspy.clients.fdsn import Client
from obspy import UTCDateTime
t1 = UTCDateTime(2023,2,6)
t2 = UTCDateTime(2023,2,7)
cl = Client("ISC")
cat = cl.get_events(minmagnitude=7.7,starttime=t1,endtime=t2,includearrivals=True)
event = cat[0]

pp = event.picks[:5]
print(pp)

[Pick(resource_id=ResourceIdentifier(id="smi:ISC/pickid=1118460649"), time=UTCDateTime(2023, 2, 6, 1, 20, 23, 300000), waveform_id=WaveformStreamID(network_code='IR', station_code='VRI'), phase_hint='P', evaluation_mode='automatic'), Pick(resource_id=ResourceIdentifier(id="smi:ISC/pickid=1118460650"), time=UTCDateTime(2023, 2, 6, 1, 20, 26, 200000), waveform_id=WaveformStreamID(network_code='IR', station_code='MLR'), phase_hint='P', evaluation_mode='automatic'), Pick(resource_id=ResourceIdentifier(id="smi:ISC/pickid=1118460651"), time=UTCDateTime(2023, 2, 6, 1, 20, 39, 900000), waveform_id=WaveformStreamID(network_code='IR', station_code='LOT'), phase_hint='P', evaluation_mode='automatic'), Pick(resource_id=ResourceIdentifier(id="smi:ISC/pickid=1118460652"), time=UTCDateTime(2023, 2, 6, 1, 20, 48, 200000), waveform_id=WaveformStreamID(network_code='IR', station_code='GZR'), phase_hint='P', evaluation_mode='automatic'), Pick(resource_id=ResourceIdentifier(id="smi:ISC/pickid=1118460653")

In [5]:
print(type(event.magnitudes))
print(type(event.magnitudes[0]))
print(event.magnitudes[0])

<class 'list'>
<class 'obspy.core.event.magnitude.Magnitude'>
Magnitude
	    resource_id: ResourceIdentifier(id="smi:ISC/magid=628419735")
	            mag: 6.8 [uncertainty=0.04]
	 magnitude_type: 'mb'
	      origin_id: ResourceIdentifier(id="smi:ISC/origid=620015154")
	  station_count: 256
	  creation_info: CreationInfo(author='NEIC')


In [6]:
# try event.<Tab> to get an idea what "children" elements event has
event.preferred_magnitude()

Magnitude
	    resource_id: ResourceIdentifier(id="smi:ISC/magid=628419735")
	            mag: 6.8 [uncertainty=0.04]
	 magnitude_type: 'mb'
	      origin_id: ResourceIdentifier(id="smi:ISC/origid=620015154")
	  station_count: 256
	  creation_info: CreationInfo(author='NEIC')

- The Catalog object contains some convenience methods to make
working with events easier.
- for example, the included events can be filtered with various keys.

In [7]:
largest_magnitude_events = catalog.filter("magnitude >= 8.0")
print(largest_magnitude_events)

1 Event(s) in Catalog:
2011-03-11T05:46:23.200000Z | +38.296, +142.498 | 9.1  MW


- There is a basic preview plot using the matplotlib basemap module.

In [8]:
catalog.plot(projection="local");

KeyboardInterrupt: 

- a (modified) Catalog can be output to file (currently there is write support for QuakeML only)

In [ ]:
largest_magnitude_events.write("/tmp/large_events.xml", format="QUAKEML")
!ls -l /tmp/large_events.xml

- the event type classes can be used to build up Events/Catalogs/Picks/.. from scratch in custom processing work flows and to share them with other researchers in the de facto standard format QuakeML

In [ ]:
from obspy import UTCDateTime
from obspy.core.event import Catalog, Event, Origin, Magnitude
from obspy.geodetics import FlinnEngdahl

cat = Catalog()
cat.description = "Just a fictitious toy example catalog built from scratch"

e = Event()
e.event_type = "not existing"

o = Origin()
o.time = UTCDateTime(2014, 2, 23, 18, 0, 0)
o.latitude = 47.6
o.longitude = 12.0
o.depth = 10000
o.depth_type = "operator assigned"
o.evaluation_mode = "manual"
o.evaluation_status = "preliminary"
o.region = FlinnEngdahl().get_region(o.longitude, o.latitude)

m = Magnitude()
m.mag = 7.2
m.magnitude_type = "Mw"

m2 = Magnitude()
m2.mag = 7.4
m2.magnitude_type = "Ms"

# also included could be: custom picks, amplitude measurements, station magnitudes,
# focal mechanisms, moment tensors, ...

# make associations, put everything together
cat.append(e)
e.origins = [o]
e.magnitudes = [m, m2]
m.origin_id = o.resource_id
m2.origin_id = o.resource_id

print(cat)
cat.write("/tmp/my_custom_events.xml", format="QUAKEML")
!cat /tmp/my_custom_events.xml

In [ ]:
cat = read_events('/tmp/my_custom_events.xml')
print(cat)
cat.plot()
